# **[HW2] Training Neural Network**
1. Prerequisite
2. Activation
3. Optimizer
4. Regularization
5. FC vs Conv
6. Do it by yourself

이번 실습에서는 지난 시간에 배웠던 MLP-layer의 component들을 하나씩 바꿔가며 activation, optimizer, regularization, convolution layer등의 중요성을 하나씩 익혀가는 시간을 갖도록 하겠습니다.  

# 1. Prerequisite

본격적인 실습을 진행하기 이전, 지난 [HW1.2 Logistic Regression vs MLP]에서 진행했던것과 동일하게 \\
Mnist dataset에 대해서 DataLoader와 Trainer class를 생성해두겠습니다.



## Import packages

런타임의 유형을 변경해줍니다.

상단 메뉴에서 [런타임]->[런타임유형변경]->[하드웨어가속기]->[GPU]

변경 이후 아래의 cell을 실행 시켰을 때, torch.cuda.is_avialable()이 True가 나와야 합니다.



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils import data
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cu113
True


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

## Load Dataset

In [3]:
mnist = fetch_openml('mnist_784', cache=False)
X = mnist.data.astype('float32').values
y = mnist.target.astype('int64').values
X /= 255.0
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


## Split Dataset

학습과 평가를 위한 dataset으로 나눕니다.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(56000, 784)
(56000,)
(14000, 784)
(14000,)


## Pytorch Dataset 

In [5]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        super(CustomDataset, self).__init__()
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]
        x = torch.from_numpy(x).float()
        y = torch.from_numpy(np.array(y)).long()
        return x, y

    def __len__(self):
        return len(self.X)

In [6]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

print(len(train_dataset))
print(train_dataset.X.shape)
print(len(test_dataset))
print(test_dataset.X.shape)

56000
(56000, 784)
14000
(14000, 784)


## DataLoader


In [7]:
batch_size = 64

# shuffle the train data
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# do not shuffle the val & test data
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# dataset size // batch_size
print(len(train_dataloader))
print(len(test_dataloader))

875
219


## Trainer


In [8]:
class Trainer():
    def __init__(self, trainloader, testloader, model, optimizer, criterion, device):
        """
        trainloader: train data's loader
        testloader: test data's loader
        model: model to train
        optimizer: optimizer to update your model
        criterion: loss function
        """
        self.trainloader = trainloader
        self.testloader = testloader
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        
    def train(self, epoch = 1):
        self.model.train()
        for e in range(epoch):
            running_loss = 0.0  
            for i, data in enumerate(self.trainloader, 0): 
                inputs, labels = data 
                # model에 input으로 tensor를 gpu-device로 보낸다
                inputs = inputs.to(self.device)  
                labels = labels.to(self.device)
                # zero the parameter gradients
                self.optimizer.zero_grad()    
                # forward + backward + optimize
                outputs = self.model(inputs) 
                loss = self.criterion(outputs, labels)  
                loss.backward() 
                self.optimizer.step() 
                running_loss += loss.item()
            
            print('epoch: %d  loss: %.3f' % (e + 1, running_loss / len(self.trainloader)))
            running_loss = 0.0
        
    def test(self):
        self.model.eval() 
        correct = 0
        for inputs, labels in self.testloader:
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)
            output = self.model(inputs) 
            pred = output.max(1, keepdim=True)[1] # get the index of the max 
            correct += pred.eq(labels.view_as(pred)).sum().item()
        test_acc = correct / len(self.testloader.dataset)
        print('test_acc: %.3f' %(test_acc))

# 2. Activation Function

이번 section에서는 가장 대표적으로 사용되는 sigmoid function과 relu function을 사용해보고 비교해보도록 하겠습니다.

![](https://drive.google.com/uc?export=view&id=1xfJBd9v9L_RgXGf8urNrYpb40zXU6gea)


- input: 784
- hidden: 32 or (32, 32)
- output: 10
- **activation: sigmoid or relu**
- optimizer: sgd
- loss: cross-entropy

## 2-layer Network + Sigmoid

In [9]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=32, 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [10]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch: 1  loss: 2.193
epoch: 2  loss: 1.820
epoch: 3  loss: 1.384
epoch: 4  loss: 1.084
epoch: 5  loss: 0.895
epoch: 6  loss: 0.769
epoch: 7  loss: 0.679
epoch: 8  loss: 0.613
epoch: 9  loss: 0.563
epoch: 10  loss: 0.524


In [11]:
trainer.test()

test_acc: 0.878


## 2-layer Network + ReLU

In [12]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=32, 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

In [13]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 1.278
epoch: 2  loss: 0.519
epoch: 3  loss: 0.405
epoch: 4  loss: 0.360
epoch: 5  loss: 0.335
epoch: 6  loss: 0.318
epoch: 7  loss: 0.304
epoch: 8  loss: 0.292
epoch: 9  loss: 0.282
epoch: 10  loss: 0.272


In [14]:
trainer.test()

test_acc: 0.923


#### Q1. Activation Function에 따라 성능의 차이가 있나요? 있다면, 왜 차이가 발생했을까요?




렐루함수는 양수일때 그값을 함수값으로 가지고 음수일때 0을 가진다. 시그모이드의 경우 원점에서 멀때 기울기가 현저히 떨어진다. 시그모이드는 학습할때 gradient가 희미해지는 특징이 있다. 이는 밑에 레이어가 더 깊어질 수록 심해져서 퍼포먼스가 렐루함수보다 차이가 심해질 것이다.

## 3-layer Network + Sigmoid

In [15]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [16]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 2.301
epoch: 2  loss: 2.296
epoch: 3  loss: 2.290
epoch: 4  loss: 2.282
epoch: 5  loss: 2.269
epoch: 6  loss: 2.243
epoch: 7  loss: 2.194
epoch: 8  loss: 2.099
epoch: 9  loss: 1.946
epoch: 10  loss: 1.760


In [17]:
trainer.test()

test_acc: 0.469


## 3-layer Network + ReLU

In [18]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [19]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 1.755
epoch: 2  loss: 0.594
epoch: 3  loss: 0.420
epoch: 4  loss: 0.367
epoch: 5  loss: 0.338
epoch: 6  loss: 0.317
epoch: 7  loss: 0.300
epoch: 8  loss: 0.286
epoch: 9  loss: 0.272
epoch: 10  loss: 0.260


In [20]:
trainer.test()

test_acc: 0.924


#### Q2. Activation function 별로 Layer 수를 늘리는 것이 성능이 어떻게 변하나요? 양상이 다르게 나타난다면 왜 그럴까요?

시그모이드는 레이어가 깊어질 수록 성능이 떨어진다. Q1에서 언급했듯이 시그모이드는 back할때때 0과 1사이 값이 계속 곱해져서 결국국 기울기값이 희미해지는 특징 때문인것 같다.


#### Q3. Activation function이 존재하지 않는다면 어떤 일이 일어날까요?

히든 레이어를 쌓는 의미가 없어집니다.f(x)=w*x 인 히든 레이어로 fff(x)를 구해볼때 w*w*w*x 가 되서 결국은 (w^3)*x 로 하나의 레이어를 추가한것과
다를바가 없기 때문입니다.

# 3. Optimization

이번 section에서는 sgd, momentum, Adam등의 optimizer를 사용해보고 성능을 비교해보도록 하겠습니다.

![](https://drive.google.com/uc?export=view&id=1xfCTx8xj4zoaombrK2bSN9nv0Z3r95jp)


- input: 784
- hidden: (32, 32)
- output: 10
- activation: relu
- **optimizer: sgd or momentum or adam**
- loss: cross-entropy

In [21]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

## 3-layer Network + ReLU + SGD

In [22]:
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [23]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 1.745
epoch: 2  loss: 0.630
epoch: 3  loss: 0.431
epoch: 4  loss: 0.373
epoch: 5  loss: 0.341
epoch: 6  loss: 0.318
epoch: 7  loss: 0.300
epoch: 8  loss: 0.285
epoch: 9  loss: 0.273
epoch: 10  loss: 0.261


In [24]:
trainer.test()

test_acc: 0.924


## 3-layer Network + ReLU + Momentum





In [25]:
model = MLP()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.99)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [26]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.595
epoch: 2  loss: 0.265
epoch: 3  loss: 0.215
epoch: 4  loss: 0.200
epoch: 5  loss: 0.183
epoch: 6  loss: 0.178
epoch: 7  loss: 0.164
epoch: 8  loss: 0.143
epoch: 9  loss: 0.160
epoch: 10  loss: 0.145


In [27]:
trainer.test()

test_acc: 0.948


## 3-layer Network + ReLU + Adam



In [28]:
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [29]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.291
epoch: 2  loss: 0.171
epoch: 3  loss: 0.146
epoch: 4  loss: 0.135
epoch: 5  loss: 0.124
epoch: 6  loss: 0.119
epoch: 7  loss: 0.112
epoch: 8  loss: 0.108
epoch: 9  loss: 0.106
epoch: 10  loss: 0.107


In [30]:
trainer.test()

test_acc: 0.957


#### Q4. Optimizer 별로 수렴 속도가 어떻게 다른가요? 
##### Q4.1 수렴 속도가 다르다면 sgd와 momentum의 차이는 왜 발생할까요? 
##### Q4.2 수렴 속도가 다르다면 momentum과 Adam의 차이는 왜 발생할까요?

1. 모멘텀은 이전 변화량을 부분적으로 기억해서 다음 변화할때 이를 반영한다. loss가 비교적 큰 폭으로 줄었음을 알 수 있다. 
2. 아담의 경우 이전까지의 변화량들을 먼 변화량일 수록 적게 비율을 부과하는
rmsprop의 방식과 모멘텀의 방식을 합친 방식이다. 모멘텀은 loss가 대체로는 작아지지만 말년에 커지기도 한다. adam을 사용하면 이를 좀 완화할 수 있다.

## 4. Regularization

이번 section에서는 image data에서 주로 사용되는 batch-normalization을 어떻게 사용하는지를 확인해보겠습니다.

![](https://drive.google.com/uc?export=view&id=1xZSWZiSxuGZAsonghidhTSfUEYiuxRtN)

- input: 784
- hidden: 32 or (32, 32)
- output: 10
- activation: relu
- optimizer: adam
- **regularizer: batch_norm**
- loss: cross-entropy

## 3-layer Network + ReLU + Adam + batch_norm

In [31]:
class MLP(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 hidden_dim=(32,32), 
                 output_dim=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim[0])
        self.bn1 = nn.BatchNorm1d(hidden_dim[0])
        self.fc2 = nn.Linear(hidden_dim[0], hidden_dim[1])
        self.bn2 = nn.BatchNorm1d(hidden_dim[1])
        self.fc3 = nn.Linear(hidden_dim[1], output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

In [32]:
model = MLP()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

MLP(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)

In [33]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.265
epoch: 2  loss: 0.144
epoch: 3  loss: 0.119
epoch: 4  loss: 0.105
epoch: 5  loss: 0.095
epoch: 6  loss: 0.090
epoch: 7  loss: 0.081
epoch: 8  loss: 0.075
epoch: 9  loss: 0.075
epoch: 10  loss: 0.070


In [34]:
trainer.test()

test_acc: 0.970


In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

26634

#### Q5. Batch-normalization을 사용하기 전 후로 성능이 어떻게 변화했나요? 왜 이러한 변화가 일어났을까요?


배치놈을 사용하면 지그재그로 가지않고 비교적 정방향으로 loss가 빠르게 줄어든다.
성능이 좋아졌습니다.



# 5. Fully-Connected Layer vs Convolution Layer

지금까지 model의 다양한 node를 바꿔가며 mnist의 성능 변화를 확인해보는 실습을 진행해 보았습니다. \\
비록, fully-connected network가 mnist 데이터에서 높은 성능을 내는데는 문제가 없었지만, 모든 layer를 fully-connected layer로 만드는 것은 엄청난 파라미터와 연산량을 필요로 하기 때문에 더욱 큰 고화질의 이미지 데이터를 처리하는데는 적합하지 않습니다. \\ 

따라서, 이번 section에서는 이미지 데이터 처리에 주로 사용되는 convolution layer를 사용해보고 파라미터 수와 성능이 어떻게 변화하는지 확인해보도록 하겠습니다. 

## Convolution Operation

![](https://drive.google.com/uc?export=view&id=1xdjTf4ab0P8qfu_TaLJ4TZzt5sk3twS6)


### Q6. Input이 (H, W, C) 일 때, stride S의 2개의 (F * F) convolutional filter를 적용하면 output이 어떻게 되나요?

필터를 두개 사용했으므로 채널은 2가 될것입니니다.
위 그림에서 5x5가 3x3을 만나 3x3 : (5-3) // 1 + 1가 되었는데
일반화하면
((H-F)//S +1 , (W-F)//S +1, 2) 가 됩니다.

In [36]:
class Conv(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 output_dim=10):
        super(Conv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=8,
                               kernel_size=7,
                               stride=2)
        self.conv2 = nn.Conv2d(in_channels=8,
                               out_channels=8,
                               kernel_size=7,
                               stride=2)
        self.fc = nn.Linear(3*3*8, output_dim)

    def forward(self, x):
        # should reshape data into image
        x = x.reshape(-1, 1, 28, 28)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.reshape(-1, 3*3*8)
        x = self.fc(x)
        return x

In [37]:
model = Conv()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

Conv(
  (conv1): Conv2d(1, 8, kernel_size=(7, 7), stride=(2, 2))
  (conv2): Conv2d(8, 8, kernel_size=(7, 7), stride=(2, 2))
  (fc): Linear(in_features=72, out_features=10, bias=True)
)

In [38]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.203
epoch: 2  loss: 0.089
epoch: 3  loss: 0.079
epoch: 4  loss: 0.077
epoch: 5  loss: 0.077
epoch: 6  loss: 0.076
epoch: 7  loss: 0.075
epoch: 8  loss: 0.075
epoch: 9  loss: 0.073
epoch: 10  loss: 0.070


In [39]:
trainer.test()

test_acc: 0.969


In [40]:
count_parameters(model)

4274

##### Q7. covolution operation은 image데이터를 다루는데 있어서 fully-connected layer에 비해 어떤 점에서 효과적일까요?


필터를 사용하면서 뭔가 엣지나 어떤 이미지 만의 경향성을
유지하기 쉬워집니다. 풀링으로 더 효과적으로 유지됩니다.

## 6. Do It By Yourself

위에서 했던 실습들과 수업에 배웠던 다양한 network component들을 참조해서 20,000개 이하의 파라미터로 98%의 accuracy를 달성해보세요!

알렉스넷을 흉내내보았습니다.

In [134]:
class CustomModel(nn.Module):
    def __init__(self, 
                 input_dim=784, 
                 output_dim=10):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels=8,
                               kernel_size=5,
                               stride=1)
        

        self.pool1 = nn.Conv2d(in_channels=8,out_channels=8,kernel_size=3,stride=2)
        self.bn1 = nn.BatchNorm2d(8)


        self.conv2 = nn.Conv2d(in_channels=8,
                               out_channels=8,
                               kernel_size=5,
                               stride=1)
        
        self.pool2 = nn.Conv2d(in_channels=8,out_channels=8,kernel_size=2,stride=1)
        self.bn2 = nn.BatchNorm2d(8)

        
        self.conv3 = nn.Conv2d(in_channels=8,
                               out_channels=8,
                               kernel_size=4,
                               stride=1)

        self.fc = nn.Linear(3*3*8, output_dim)
    def forward(self, x):
        # should reshape data into image
        x = x.reshape(-1, 1, 28, 28)
        x = self.conv1(x)
        x = F.relu(x)

        x=self.pool1(x)
        x=self.bn1(x)
        x = F.relu(x)


        x = self.conv2(x)
        x = F.relu(x)
  
        x=self.pool2(x)
        x=self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)

        x = x.reshape(-1, 3*3*8)
        x = self.fc(x)
        
        return x

In [135]:
model = CustomModel()
if count_parameters(model) > 20000:
  raise AssertionError

In [136]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model.to(device)

CustomModel(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool1): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 2))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1))
  (pool2): Conv2d(8, 8, kernel_size=(2, 2), stride=(1, 1))
  (bn2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(8, 8, kernel_size=(4, 4), stride=(1, 1))
  (fc): Linear(in_features=72, out_features=10, bias=True)
)

In [137]:
trainer = Trainer(trainloader = train_dataloader,
                  testloader = test_dataloader,
                  model = model,
                  criterion = criterion,
                  optimizer = optimizer,
                  device = device)

trainer.train(epoch = 10)

epoch: 1  loss: 0.185
epoch: 2  loss: 0.084
epoch: 3  loss: 0.071
epoch: 4  loss: 0.064
epoch: 5  loss: 0.061
epoch: 6  loss: 0.056
epoch: 7  loss: 0.052
epoch: 8  loss: 0.053
epoch: 9  loss: 0.050
epoch: 10  loss: 0.048


In [138]:
trainer.test()

test_acc: 0.982
